In [16]:
import time as t
import datetime as dt
import pandas as pd
import os
import sys
sys.path.insert(1, '../')
from utils.common import *
from utils.blockchain import *

In [17]:
TAKE_PROFIT = 1
STOP_LOSS = 1
PRIORITY_FEE = 5000

In [69]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT *  FROM calls"
    db_tokens = pd.read_sql_query(query, conn)
db_tokens

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,decision,buy,buy_time,buy_price
0,231763,2024-05-01 06:31:27,7UgdhWXE4yEssEtHYAEinEQEMSLnK1wVX4csxs4P9NBN,11 seconds ago,2024-05-01 06:31:16,1,0,0,1,1,12380,24760,1.0,2024-05-01 06:31:00+00:00,Test 1 not passed,None,None,None
1,231764,2024-05-01 06:34:14,CZsg66EKjJvUhQ6EsMLWkA14KYhQ1DXozga2Kh9Kp1cA,In 2 seconds,2024-05-01 06:34:16,0,0,1,0,1,1240,2480,1.0,2024-05-01 06:34:00+00:00,Test 1 not passed,None,None,None
2,231765,2024-05-01 06:35:01,bHYWGZKY7eKJx65AqfryqFfhJWsHr2i541YYwV8L6yE,27 seconds ago,2024-05-01 06:34:34,1,0,0,0,2,30,61,1.0,2024-05-01 06:39:00+00:00,Test 1 not passed,None,None,None
3,231766,2024-05-01 06:35:37,99qJN4215TwNEUyVsQEnc1D9ahLcdddcqkY7F11LC4sg,19 seconds ago,2024-05-01 06:35:18,1,1,0,0,2,48290,96570,1.0,2024-05-01 06:35:00+00:00,Test 1 not passed,None,None,None
4,231767,2024-05-01 06:35:39,Hfn4CW9MsQd1BG7cbtMn3FKkmxV5V3oEkmc2qWXQAaFF,26 seconds ago,2024-05-01 06:35:13,1,1,0,0,2,43330,86670,1.0,2024-05-01 06:35:00+00:00,Test 1 not passed,None,None,None
5,231768,2024-05-01 06:39:25,67oPEZyFu2AoGWVJoBmBD1C8utpUfWAcqod4LXJUqS46,14 seconds ago,2024-05-01 06:39:11,1,0,0,0,2,12350,24690,1.0,2024-05-01 06:39:00+00:00,Test 1 not passed,None,None,None
6,231769,2024-05-01 06:40:53,7XBTLa2KoFEFqmjauVK5Wx2oEsbHzD6bs9BYSsgHyFke,13 seconds ago,2024-05-01 06:40:40,1,1,0,0,2,59260,118510,1.0,2024-05-01 06:40:00+00:00,Test 1 not passed,None,None,None
7,231770,2024-05-01 06:43:20,14NmT6VLGU35GQA3tFp8hz9hG8UJwA1Xs3VBFp75o9Yo,In 1 minute,2024-05-01 06:44:20,0,0,0,0,1,774,1240,0.0,None,None,None,None,None
8,231771,2024-05-01 06:44:39,J1fK7Vn5Vamwjr7XvPfnCpi1UYhvrKZavA1d9PnvA8sv,3 seconds ago,2024-05-01 06:44:36,0,1,0,0,1,123910,247820,1.0,2024-05-01 06:44:00+00:00,Test 1 not passed,None,None,None
9,231772,2024-05-01 06:45:11,G12x2u53GDdsJrx1rBCVrnACcXK54htSJzAUytV84Fny,476262 hours and 45 minutes ago,2024-05-01 06:45:11,1,1,1,0,3,619,1240,0.0,None,None,None,None,None


In [18]:
tow = getSPLtokens(wallet)

In [19]:
tow

,pubkey,account,token_address,decimals,amount,amount_int
0,9bTDu83hE37NSCvkNrXfCyNLHzQc7rV8XF7KyqqzaUQ8,"{'lamports': 2039280, 'data': {'program': 'spl...",EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,6,19820014,19.820014
1,5qRpRknfKus1qiNL1iQU92atWtktTsnBMoHppLfMzinP,"{'lamports': 2039280, 'data': {'program': 'spl...",AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,9,5938092146082,5938.092146
2,7izYUZtuGMoXCC8Wx2uHpX8Ui4png7X6S64WnLs36nFQ,"{'lamports': 2039280, 'data': {'program': 'spl...",8AntP7Hxft8Yjr52UQ3RnBHU4njTkDFDho97gE4vRzkt,9,902936636325,902.936636


In [20]:
tracked_tokens = pd.merge(
    tow,
    db_tokens[['address','buy_price']],
    how='inner', left_on='token_address', right_on='address'
)
cur_prices = check_multi_price(tracked_tokens['token_address'].to_list())
cur_prices = pd.DataFrame().from_dict(cur_prices,orient='index')
cur_prices = cur_prices.reset_index().rename(columns={'index':'address'})
tracked_tokens = pd.merge(tracked_tokens, cur_prices, how='inner', on='address')
tracked_tokens['liq_req'] = tracked_tokens['amount_int']*tracked_tokens['price']
tracked_tokens['price_change'] = tracked_tokens['price']/tracked_tokens['buy_price']
tracked_tokens['sell_tag'] = np.where((tracked_tokens['liq_req'] < tracked_tokens['liquidity']) &
                                      ((tracked_tokens['price_change']>=TAKE_PROFIT) |
                                       (tracked_tokens['price_change']<-STOP_LOSS)), True, False)

In [25]:
print(tracked_tokens[['address','price_change']])

                                        address  price_change
0  AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy      0.994101


In [11]:
for i, row in tracked_tokens[tracked_tokens['sell_tag']==True].iterrows():
    print(dt.datetime.now(), 'Attempt to SELL: ', i, row['address'], '. X:', row['price_change'])
    result, txid = tx_procedure(wallet=wallet, asset_in=row['address'], asset_out=SOL_ca,
                                mode='sell', fee=PRIORITY_FEE)
    print('Result: ', result)
    if result == {'Ok': None}:
        print('SUCCESS BUY')
        ## UPDATE TO DB
        with SQLiteDB('dbs/calls.db') as conn:
            update_statement = f"""
            UPDATE calls
            SET buy = {False},
            WHERE address = "{row['address']}"
            """
            conn.execute(update_statement)
        print('DB updated with sell data!')

2024-04-28 19:35:57.484823 Attempt to SELL:  0 AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy
attempt to sell 101191.563343478 of AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy
Success Quote!
fee will be:  5000
Success Txgen!
Success sign!


RuntimeError: asyncio.run() cannot be called from a running event loop